In [1]:
import sys
sys.path.append('../')
import numpy as np
import matplotlib.pyplot as plt

import PyCapture2 as pc2
import devices.Camera
import daq
import file
%load_ext autoreload
%autoreload 1

In [2]:
%aimport devices.Camera
cam = devices.Camera.Camera(17529184)

Fc2error: b'Bus Master Failure.'

In [ ]:
#PROP = pc2.PROPERTY_TYPE.SHARPNESS
#PROP = pc2.PROPERTY_TYPE.BRIGHTNESS
#PROP = pc2.PROPERTY_TYPE.AUTO_EXPOSURE
PROP = pc2.PROPERTY_TYPE.BRIGHTNESS
prop = cam.cam.getProperty(PROP)
info = cam.cam.getPropertyInfo(PROP)
print('Present:', info.present)
print('Auto supported:', info.autoSupported)
print('Manual supported:', info.manualSupported)
print('Units:', info.units)
print('onOff:', prop.onOff)
print('autoManual:', prop.autoManualMode)
print('Min:', info.absMin)
print('absValue:', prop.absValue)
print('Max:', info.absMax)
print('Min:', info.min)
print('Value:', prop.valueA)
print('Max:', info.max)

In [ ]:
cam.close()

In [ ]:
info = cam.cam.getGigEImageSettings()
info.pixelFormat

In [ ]:
pc2.PIXEL_FORMAT.RAW16

In [ ]:
image = cam.take_photo()

In [ ]:
print(len(image.getData())/(2048*1536))
print(image.getData()[0:100])

In [ ]:
%timeit image.save(b'/media/robert/Data_Storage/daq/test.tiff', pc2.IMAGE_FILE_FORMAT.TIFF)

In [ ]:
tiff = plt.imread('/media/robert/Data_Storage/daq/test.tiff')
plt.imshow(tiff/16)
plt.colorbar()
plt.show()

In [ ]:
from PIL import Image
im = Image.open('/media/robert/Data_Storage/daq/test.tiff')
im.info

In [ ]:
shutPROP = pc2.PROPERTY_TYPE.SHUTTER
shut = cam.cam.getProperty(shutPROP)
print(shut)

In [ ]:
cam.cam.disconnect()

In [ ]:
daq.main([['Camera'], [17571186], 'cam_test', 10])

In [ ]:
from PIL import Image
dataSet = 1806150049

image = file.load_IMAGE(17583372, dataSet, 10)
print(file.decode_image_meta(image))

In [ ]:
tiff = plt.imread('/media/robert/Data_Storage/daq/IMAGE/year_2018/month_06/day_15/1806150049/17583372_1806150049_0010.tiff')
plt.imshow(tiff)
plt.colorbar()
plt.show()

In [ ]:
import time
def callback(image, i):
    image.save(bytes('test.tiff', 'utf-8'), pc2.IMAGE_FILE_FORMAT.TIFF)
    print(time.clock())
#cam.cam.startCapture(callback)
cam.start_capture(callback)#, args=(1,))
time.sleep(1)
cam.cam.stopCapture()

In [1]:
# Cython implementation
import sys
sys.path.append('../')
import numpy as np
from calc.camera import convert_image
raw = [i % 200 for i in range(8000000)]
height = 2000
width = 2000
%timeit data = convert_image(raw, height, width)

169 ms ± 209 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [2]:
raw = [i % 200 for i in range(8000000)]
height = 2000
width = 2000
b = bytes(raw)
numpy_array = np.frombuffer(b, dtype=np.uint16)
%timeit bytes(raw)
%timeit np.frombuffer(b, dtype=np.uint16)
%timeit numpy_array.reshape(height, width)
%timeit np.frombuffer(bytes(raw), dtype=np.uint16).reshape(height, width) 

37 ms ± 8.64 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
576 ns ± 5.77 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
234 ns ± 0.845 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
37 ms ± 18.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
